In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# 数据预处理/打标签
data_url = r"effect_validation_dataset\boston.csv"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]
house_df = pd.DataFrame(data)
house_targe = pd.DataFrame(target)
house_df.columns = [['crim','zn','indus','chas','nox','rm','age','dis','rad','tax','ptradio','b','lstat']]
house_targe.columns = [['medv'] ]

# 数据集划分(训练集:测试集=7:3)
X_train, X_test, Y_train, Y_test = train_test_split(house_df, house_targe,random_state=23)

In [3]:
X_train

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptradio,b,lstat
112,0.12329,0.0,10.01,0.0,0.547,5.913,92.9,2.3534,6.0,432.0,17.8,394.95,16.21
301,0.03537,34.0,6.09,0.0,0.433,6.590,40.4,5.4917,7.0,329.0,16.1,395.75,9.50
401,14.23620,0.0,18.10,0.0,0.693,6.343,100.0,1.5741,24.0,666.0,20.2,396.90,20.32
177,0.05425,0.0,4.05,0.0,0.510,6.315,73.4,3.3175,5.0,296.0,16.6,395.60,6.29
69,0.12816,12.5,6.07,0.0,0.409,5.885,33.0,6.4980,4.0,345.0,18.9,396.90,8.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,13.67810,0.0,18.10,0.0,0.740,5.935,87.9,1.8206,24.0,666.0,20.2,68.95,34.02
457,8.20058,0.0,18.10,0.0,0.713,5.936,80.3,2.7792,24.0,666.0,20.2,3.50,16.94
40,0.03359,75.0,2.95,0.0,0.428,7.024,15.8,5.4011,3.0,252.0,18.3,395.62,1.98
230,0.53700,0.0,6.20,0.0,0.504,5.981,68.1,3.6715,8.0,307.0,17.4,378.35,11.65


In [12]:
Y_train

,medv
112,18.8
301,22.0
401,7.2
177,24.6
69,20.9
...,...
438,8.4
457,13.5
40,34.9
230,24.3


In [14]:
X_test

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptradio,b,lstat
176,0.07022,0.0,4.05,0.0,0.510,6.020,47.2,3.5549,5.0,296.0,16.6,393.23,10.11
311,0.79041,0.0,9.90,0.0,0.544,6.122,52.8,2.6403,4.0,304.0,18.4,396.90,5.98
94,0.04294,28.0,15.04,0.0,0.464,6.249,77.3,3.6150,4.0,270.0,18.2,396.90,10.59
139,0.54452,0.0,21.89,0.0,0.624,6.151,97.9,1.6687,4.0,437.0,21.2,396.90,18.46
232,0.57529,0.0,6.20,0.0,0.507,8.337,73.3,3.8384,8.0,307.0,17.4,385.91,2.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...
442,5.66637,0.0,18.10,0.0,0.740,6.219,100.0,2.0048,24.0,666.0,20.2,395.69,16.59
100,0.14866,0.0,8.56,0.0,0.520,6.727,79.9,2.7778,5.0,384.0,20.9,394.76,9.42
367,13.52220,0.0,18.10,0.0,0.631,3.863,100.0,1.5106,24.0,666.0,20.2,131.42,13.33
375,19.60910,0.0,18.10,0.0,0.671,7.313,97.9,1.3163,24.0,666.0,20.2,396.90,13.44


In [16]:
Y_test

,medv
176,23.2
311,22.1
94,20.6
139,17.8
232,41.7
...,...
442,18.4
100,27.5
367,23.1
375,15.0


In [22]:
class LR(object):
    def __init__(self,fit_intercept=True):
        """
        :param fit_intercept: 是否加入截距项，如果加入，需要对X第一列增广，即拼接上全1向量
        """
        self.beta = None #线性回归参数
        self.fit_intercept = fit_intercept

    def fit(self, X, y):
        """
        :param X: 样本矩阵
        """
        #当有截距项，需要在X左侧增广，加上一列全1的列向量
        if self.fit_intercept:
            X = np.hstack((np.ones_like(y.reshape((-1,1))), X))

        ##判断(XTX)是否可逆
        n_sample = X.shape[0]
        n_feature = X.shape[1]

        #简化处理：特征数量大于样本数量时矩阵显然不可逆
        if n_feature > n_sample:
            is_inv = False
        #进一步判断行列式
        elif np.linalg.det(np.matmul(X.T, X)) == 0:
            is_inv = False
        else:
            is_inv = True

        #可逆
        if is_inv:
            self.beta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        #不可逆
        else:
            u,s,vt = np.linalg.svd(X) #SVD分解,s是向量，表示奇异值
            if len(np.nonzero(s)[0]) == X.shape[0]:
                sigma_inv_vector = 1 / s
            else:#当出现0奇异值，1/s会报错，另外处理
                n_nonzero = len(np.nonzero(s)[0])
                s_nonzero = s[:n_nonzero]
                s_inv = 1 / s #对角阵的伪逆
                zeros = np.zeros((n_feature - len(s_inv)))
                sigma_inv_vector = np.hstack((s_inv,zeros))
            sigma_inv_diag = np.diag(sigma_inv_vector)
            if X.shape[0] == X.shape[1]: #sigma是方阵(行=列)
                sigma_inv = sigma_inv_diag
            elif X.shape[0] > X.shape[1]: #sigma是竖的矩形(行>列)
                sigma_zeros = np.zeros((X.shape[1],(X.shape[0] - X.shape[1])))
                sigma_inv = np.hstack((sigma_inv_diag, sigma_zeros))
            else:#sigma是横的矩形(行<列)
                sigma_zeros = np.zeros(((X.shape[1] - X.shape[0]),X.shape[0]))
                sigma_inv = np.vstack((sigma_inv_diag, sigma_zeros))

            self.beta = vt.T @ sigma_inv @ u.T @ y

        self.beta = self.beta.reshape((-1,1))

    def predict(self, X):
        """
        :param X: 测试集
        :return: y_predict
        """
        if X.shape[1] != self.beta.shape[0]:
            X = np.hstack((np.ones((X.shape[0], 1)), X))

        y_predict = X.dot(self.beta)
        return y_predict


In [23]:
def cal_mse(y_predict, y_true):
    assert y_predict.ndim == y_true.ndim, 'y_predict和y_true需要维度相同'
    return np.mean((y_predict - y_true) ** 2)

In [25]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
print('=====方法1：本文方法=====')
# SELF
lr = LR(fit_intercept=True)
lr.fit(X_train.values,Y_train.values)
Y_train_predict = lr.predict(X_train.values)
Y_test_predict = lr.predict(X_test.values)
print('MSE in training set:%.4f'%(mean_squared_error(Y_train.values, Y_train_predict)))
print('MSE in testing set:%.4f' % (mean_squared_error(Y_test.values, Y_test_predict)))
print('MAE in training set:%.4f'%(mean_absolute_error(Y_train.values, Y_train_predict)))
print('MAE in testing set:%.4f' % (mean_absolute_error(Y_test.values, Y_test_predict)))
print('RMSE in training set:%.4f'%(np.sqrt(mean_absolute_error(Y_train.values, Y_train_predict))))
print('RMSE in testing set:%.4f' % (np.sqrt(mean_absolute_error(Y_test.values, Y_test_predict))))

# SK
lr_sk = LinearRegression()
lr_sk.fit(X_train.values, Y_train.values)
Y_train_predict_sk = lr_sk.predict(X_train.values)
Y_test_predict_sk = lr_sk.predict(X_test.values)
print('=====Sklearn Linear Regression=====')
print('MSE in training set:%.4f'%(mean_squared_error(Y_train.values, Y_train_predict_sk)))
print('MSE in testing set:%.4f' % (mean_squared_error(Y_test.values, Y_test_predict_sk)))
print('MAE in training set:%.4f'%(mean_absolute_error(Y_train.values, Y_train_predict_sk)))
print('MAE in testing set:%.4f' % (mean_absolute_error(Y_test.values, Y_test_predict_sk)))
print('RMSE in training set:%.4f'%(np.sqrt(mean_absolute_error(Y_train.values, Y_train_predict_sk))))
print('RMSE in testing set:%.4f' % (np.sqrt(mean_absolute_error(Y_test.values, Y_test_predict_sk))))

=====方法1：本文方法=====
MSE in training set:21.8721
MSE in testing set:22.7708
MAE in training set:3.2154
MAE in testing set:3.6535
RMSE in training set:1.7931
RMSE in testing set:1.9114
=====Sklearn Linear Regression=====
MSE in training set:21.8721
MSE in testing set:22.7708
MAE in training set:3.2154
MAE in testing set:3.6535
RMSE in training set:1.7931
RMSE in testing set:1.9114
